In [15]:
from urllib.parse import unquote
import csv
import folium
import openrouteservice
from branca.element import Figure
from openrouteservice import convert
import pandas as pd

waketa_list = []
datalist = []

with open('test.txt') as f:
    for line in f:
        waketa_list.append(line.split("/"))

for i in range(len(waketa_list)):
    chimei = unquote(waketa_list[i][5])
    zahyo_matomari = waketa_list[i][6].split(",")
    keido = zahyo_matomari[0][1:]
    ido = zahyo_matomari[1]
    datalist.append([chimei,keido,ido])
    
df_list = pd.DataFrame(datalist, columns=["地名","緯度","経度"])
df_list.to_csv("loc_pos.csv", index=False, encoding="utf_8")

In [21]:
key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
client = openrouteservice.Client(key=key)

datalist = []

with open("loc_pos.csv", encoding="utf_8") as f:
    reader = csv.reader(f)
    line = [row for row in reader]

for i in range(len(line)-2):
    p1 = float(line[i+1][1]), float(line[i+1][2])
    for j in range(i,len(line)-2):
        p2 = float(line[j+2][1]), float(line[j+2][2])
        print(p1,p2)
        p1r = tuple(reversed(p1))
        p2r = tuple(reversed(p2))
        mean_lat = (p1[0] + p2[0]) / 2
        mean_long = (p1[1] + p2[1]) / 2

        # 経路計算 (Directions V2)
        routedict = client.directions((p1r, p2r),profile="foot-walking")
        geom = routedict["routes"][0]["geometry"]
        decoded = convert.decode_polyline(geom)

        print(routedict["routes"][0]["summary"]["duration"])
        datalist.append([line[i+1][0],line[j+2][0],str(routedict["routes"][0]["summary"]["duration"])+"[s]"])
    
df_list = pd.DataFrame(datalist, columns=["出発地点","行先","移動時間"])
df_list.to_csv("loc_loc_time.csv", index=False, encoding="utf_8")

(35.6311616, 139.7844807) (35.6305754, 139.7903577)
495.3
(35.6311616, 139.7844807) (35.649125, 139.787199)
2037.0
(35.6305754, 139.7903577) (35.649125, 139.787199)
2023.0
